In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
from datetime import datetime, timezone, timedelta
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config
import src.config as config
from src.inference import (
    get_feature_store
)


c:\Users\singh\Downloads\CDS500_Applied_ML_DS\Projects\CDA500P1\CDA500P1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 03:00:00+00:00


In [3]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [4]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()
df= query.read()
df.head(20)
df.shape

2025-03-04 21:22:18,003 INFO: Initializing external client
2025-03-04 21:22:18,004 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:22:19,037 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.60s) 


,pickup_location_id,predicted_demand,pickup_hour
0,229,30.0,2025-03-05 00:00:00+00:00
1,85,0.0,2025-03-05 00:00:00+00:00
2,36,0.0,2025-03-05 00:00:00+00:00
3,252,0.0,2025-03-05 00:00:00+00:00
4,133,0.0,2025-03-05 00:00:00+00:00
5,80,1.0,2025-03-05 00:00:00+00:00
6,24,9.0,2025-03-05 00:00:00+00:00
7,180,0.0,2025-03-05 00:00:00+00:00
8,92,0.0,2025-03-05 00:00:00+00:00
9,203,0.0,2025-03-05 00:00:00+00:00


(753, 3)

In [5]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-04 21:22:30,348 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:22:30,355 INFO: Initializing external client
2025-03-04 21:22:30,356 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:22:30,962 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689
Current UTC time: 2025-03-05 02:22:30.348438+00:00
Next hour: 2025-03-05 03:00:00+00:00
Querying for date range: 2025-03-05 to 2025-03-06
Filtering for hour: 2025-03-05 03:00
Found 251 records


In [6]:
now = datetime.now(timezone.utc)
predictions.head(50)
predictions.shape


,pickup_location_id,predicted_demand,pickup_hour
502,181,1.0,2025-03-05 03:00:00+00:00
503,153,0.0,2025-03-05 03:00:00+00:00
504,252,0.0,2025-03-05 03:00:00+00:00
505,223,1.0,2025-03-05 03:00:00+00:00
506,124,0.0,2025-03-05 03:00:00+00:00
507,37,0.0,2025-03-05 03:00:00+00:00
508,125,18.0,2025-03-05 03:00:00+00:00
509,93,0.0,2025-03-05 03:00:00+00:00
510,152,3.0,2025-03-05 03:00:00+00:00
511,251,0.0,2025-03-05 03:00:00+00:00


(251, 3)

In [7]:
df.info()
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 753 entries, 0 to 752
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  753 non-null    int32                  
 1   predicted_demand    753 non-null    float64                
 2   pickup_hour         753 non-null    datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 14.8 KB


In [8]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-04 21:22:32,153 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:22:32,157 INFO: Initializing external client
2025-03-04 21:22:32,157 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:22:32,834 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.56s) 


In [9]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()
results.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (8.85s) 


,pickup_location_id,predicted_demand,pickup_hour


,pickup_location_id,predicted_demand,pickup_hour
0,229,30.0,2025-03-05 00:00:00+00:00
1,85,0.0,2025-03-05 00:00:00+00:00
2,36,0.0,2025-03-05 00:00:00+00:00
3,252,0.0,2025-03-05 00:00:00+00:00
4,133,0.0,2025-03-05 00:00:00+00:00


In [10]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

predictions = fetch_next_hour_predictions()

2025-03-04 21:22:45,625 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:22:45,632 INFO: Initializing external client
2025-03-04 21:22:45,633 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:22:46,371 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689
Current UTC time: 2025-03-05 02:22:45.625680+00:00
Next hour: 2025-03-05 03:00:00+00:00
Found 251 records


In [11]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)
df = fg.read()
df["pickup_hour"].max()
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)
df[df["pickup_hour"] == next_hour]

2025-03-04 21:22:47,342 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 21:22:47,348 INFO: Initializing external client
2025-03-04 21:22:47,349 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 21:22:48,025 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214689
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.51s) 


Timestamp('2025-03-05 03:00:00+0000', tz='Etc/UTC')

2025-03-05 03:00:00+00:00


,pickup_location_id,predicted_demand,pickup_hour
502,181,1.0,2025-03-05 03:00:00+00:00
503,153,0.0,2025-03-05 03:00:00+00:00
504,252,0.0,2025-03-05 03:00:00+00:00
505,223,1.0,2025-03-05 03:00:00+00:00
506,124,0.0,2025-03-05 03:00:00+00:00
...,...,...,...
748,242,0.0,2025-03-05 03:00:00+00:00
749,144,18.0,2025-03-05 03:00:00+00:00
750,29,1.0,2025-03-05 03:00:00+00:00
751,91,0.0,2025-03-05 03:00:00+00:00
